In [27]:
# =========== 第一部分：导入所有依赖库 ===========
import os
import csv
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mindspore as ms
import mindspore.dataset as ds
from mindspore.dataset import GeneratorDataset
import mindspore.nn as nn
from mindspore import Tensor

# 设置MindSpore运行环境
ms.set_context(mode=ms.GRAPH_MODE, device_target="GPU")
print("✅ 依赖库导入完成！")
print(f"MindSpore版本：{ms.__version__}")

✅ 依赖库导入完成！
MindSpore版本：1.7.0


In [28]:
# =========== 第二部分：下载数据集 ===========
# 设置代理环境变量
import os
os.environ['no_proxy'] = 'a.test.com,127.0.0.1,2.2.2.2'

# 下载数据集
!wget https://ascend-professional-construction-dataset.obs.cn-north-4.myhuaweicloud.com/deep-learning/auto-mpg.data

print("✅ 数据集下载完成！")

--2025-12-10 15:00:53--  https://ascend-professional-construction-dataset.obs.cn-north-4.myhuaweicloud.com/deep-learning/auto-mpg.data
Resolving proxy.modelarts.com (proxy.modelarts.com)... 192.168.6.3
Connecting to proxy.modelarts.com (proxy.modelarts.com)|192.168.6.3|:80... connected.
Proxy request sent, awaiting response... 200 OK
Length: 30286 (30K) [binary/octet-stream]
Saving to: ‘auto-mpg.data.1’

auto-mpg.data.1     100%[===================>]  29.58K  --.-KB/s    in 0.001s  

2025-12-10 15:00:53 (21.9 MB/s) - ‘auto-mpg.data.1’ saved [30286/30286]

✅ 数据集下载完成！


In [29]:
# ==================== 第三部分：查看原始数据格式 ====================
print("📊 查看原始数据格式")
print("="*50)

with open('auto-mpg.data') as csv_file:
    data_lines = list(csv.reader(csv_file, delimiter=','))
    
print(f"数据集总行数：{len(data_lines)}")
print("\n示例数据（第20-25行）：")
for i in range(20, 25):
    print(f"行{i+1}: {data_lines[i]}")

📊 查看原始数据格式
数据集总行数：398

示例数据（第20-25行）：
行21: ['25.0   4   110.0      87.00      2672.      17.5   70  2\t"peugeot 504"']
行22: ['24.0   4   107.0      90.00      2430.      14.5   70  2\t"audi 100 ls"']
行23: ['25.0   4   104.0      95.00      2375.      17.5   70  2\t"saab 99e"']
行24: ['26.0   4   121.0      113.0      2234.      12.5   70  2\t"bmw 2002"']
行25: ['21.0   6   199.0      90.00      2648.      15.0   70  1\t"amc gremlin"']


In [30]:
# ==================== 第四部分：使用pandas读取数据 ====================
print("📋 使用pandas读取数据")
print("="*50)

column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_data = pd.read_csv('auto-mpg.data', 
                       names=column_names,
                       na_values="?", 
                       comment='\t',
                       sep=" ", 
                       skipinitialspace=True)

data = raw_data.copy()

print(f"数据形状：{data.shape}")
print("\n数据前5行：")
print(data.head())

📋 使用pandas读取数据
数据形状：(398, 8)

数据前5行：
    MPG  Cylinders  Displacement  Horsepower  Weight  Acceleration  \
0  18.0          8         307.0       130.0  3504.0          12.0   
1  15.0          8         350.0       165.0  3693.0          11.5   
2  18.0          8         318.0       150.0  3436.0          11.0   
3  16.0          8         304.0       150.0  3433.0          12.0   
4  17.0          8         302.0       140.0  3449.0          10.5   

   Model Year  Origin  
0          70       1  
1          70       1  
2          70       1  
3          70       1  
4          70       1  


In [31]:
# ==================== 第五部分：数据清洗与特征处理 ====================
print("🧹 数据清洗与特征处理")
print("="*50)

# 删除缺失值
print(f"原始数据行数：{len(data)}")
data = data.dropna()
print(f"清洗后数据行数：{len(data)}")

# 分离标签和特征
origin = data.pop('Origin')
data_labels = data.pop('MPG')

# 数据统计信息
train_stats = data.describe()
train_stats = train_stats.transpose()
print("\n数据统计信息：")
print(train_stats)

🧹 数据清洗与特征处理
原始数据行数：398
清洗后数据行数：392

数据统计信息：
              count         mean         std     min       25%     50%  \
Cylinders     392.0     5.471939    1.705783     3.0     4.000     4.0   
Displacement  392.0   194.411990  104.644004    68.0   105.000   151.0   
Horsepower    392.0   104.469388   38.491160    46.0    75.000    93.5   
Weight        392.0  2977.584184  849.402560  1613.0  2225.250  2803.5   
Acceleration  392.0    15.541327    2.758864     8.0    13.775    15.5   
Model Year    392.0    75.979592    3.683737    70.0    73.000    76.0   

                   75%     max  
Cylinders        8.000     8.0  
Displacement   275.750   455.0  
Horsepower     126.000   230.0  
Weight        3614.750  5140.0  
Acceleration    17.025    24.8  
Model Year      79.000    82.0  


In [32]:
# ==================== 第六部分：数据归一化 ====================
print("📏 数据归一化处理")
print("="*50)

# 定义归一化函数
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_data = norm(data)
normed_data['MPG'] = data_labels

# One-hot编码
normed_data['USA'] = (origin == 1) * 1.0
normed_data['Europe'] = (origin == 2) * 1.0
normed_data['Japan'] = (origin == 3) * 1.0

print(f"处理后数据形状：{normed_data.shape}")
print(f"特征列：{list(normed_data.columns)}")

📏 数据归一化处理
处理后数据形状：(392, 10)
特征列：['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'MPG', 'USA', 'Europe', 'Japan']


In [34]:
# ==================== 第七部分：划分训练集和测试集 ====================
print(" 划分训练集和测试集")
print("="*50)

# 按80%/20%比例划分
train_dataset = normed_data.sample(frac=0.8, random_state=0)
test_dataset = normed_data.drop(train_dataset.index)

print(f"训练集大小：{len(train_dataset)} 行")
print(f"测试集大小：{len(test_dataset)} 行")

# 分离特征和标签
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

X_train = np.array(train_dataset, dtype=np.float32)
Y_train = np.array(train_labels, dtype=np.float32)
X_test = np.array(test_dataset, dtype=np.float32)
Y_test = np.array(test_labels, dtype=np.float32)

print(f"\nX_train形状：{X_train.shape}")
print(f"Y_train形状：{Y_train.shape}")
print(f"X_test形状：{X_test.shape}")
print(f"Y_test形状：{Y_test.shape}")

 划分训练集和测试集
训练集大小：314 行
测试集大小：78 行

X_train形状：(314, 9)
Y_train形状：(314,)
X_test形状：(78, 9)
Y_test形状：(78,)


In [35]:
# ==================== 第八部分：创建MindSpore数据集 ====================
print(" 创建MindSpore数据集")
print("="*50)

class Iterable:
    def __init__(self, X, Y):
        self._data = X
        self._label = Y[:, np.newaxis]
    
    def __getitem__(self, index):
        return self._data[index], self._label[index]
    
    def __len__(self):
        return len(self._data)

train_iter = Iterable(X_train, Y_train)
test_iter = Iterable(X_test, Y_test)

dataset_train = GeneratorDataset(source=train_iter, column_names=["data", "label"])
dataset_test = GeneratorDataset(source=test_iter, column_names=["data", "label"])

print(f"训练数据集大小：{dataset_train.get_dataset_size()}")
print(f"测试数据集大小：{dataset_test.get_dataset_size()}")

 创建MindSpore数据集
训练数据集大小：314
测试数据集大小：78


In [36]:
# ==================== 第九部分：构建神经网络模型 ====================
print("🧠 构建全连接神经网络")
print("="*50)

class RegressionCar(nn.Cell):
    """汽车油耗预测神经网络 - 3层全连接网络"""
    def __init__(self):
        super(RegressionCar, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Dense(9, 64, activation='relu')   # 输入层：9个特征
        self.fc2 = nn.Dense(64, 64, activation='relu')  # 隐藏层：64个神经元
        self.fc3 = nn.Dense(64, 1)                      # 输出层：1个值（MPG预测）
    
    def construct(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

# 实例化网络
network = RegressionCar()

print("神经网络结构：")
print(network)
print("\n✅ 网络构建完成！")

🧠 构建全连接神经网络
神经网络结构：
RegressionCar<
  (flatten): Flatten<>
  (fc1): Dense<
    input_channels=9, output_channels=64, has_bias=True, activation=ReLU<>
    (activation): ReLU<>
    >
  (fc2): Dense<
    input_channels=64, output_channels=64, has_bias=True, activation=ReLU<>
    (activation): ReLU<>
    >
  (fc3): Dense<input_channels=64, output_channels=1, has_bias=True>
  >

✅ 网络构建完成！


In [37]:
# ==================== 第十部分：定义损失函数和优化器 ====================
print(" 配置损失函数和优化器")
print("="*50)

# 使用文档中的配置
net_loss = nn.MSELoss()  # 均方误差损失
net_opt = nn.RMSProp(network.trainable_params(), learning_rate=0.001)  # RMSProp优化器

print(f"损失函数：{net_loss}")
print(f"优化器：RMSProp，学习率：0.001")
print(f"可训练参数数量：{len(network.trainable_params())}")

 配置损失函数和优化器
损失函数：MSELoss<>
优化器：RMSProp，学习率：0.001
可训练参数数量：6


In [38]:
# ==================== 第十一部分：定义训练函数 ====================
print("🏋️ 定义模型训练函数")
print("="*50)

def train_loop(model, dataset, loss_fn, optimizer):
    """
    训练循环函数 - 使用TrainOneStepCell（MindSpore 1.7兼容）
    参数：
        model: 神经网络模型
        dataset: 训练数据集
        loss_fn: 损失函数
        optimizer: 优化器
    返回：
        平均损失
    """
    # 创建训练网络
    class TrainNet(nn.Cell):
        def __init__(self, network, loss_fn):
            super(TrainNet, self).__init__()
            self.network = network
            self.loss_fn = loss_fn
            
        def construct(self, data, label):
            output = self.network(data)
            return self.loss_fn(output, label)
    
    # 创建训练网络和一步训练器
    train_net = TrainNet(model, loss_fn)
    train_step = nn.TrainOneStepCell(train_net, optimizer)
    
    # 设置为训练模式
    train_step.set_train(True)
    
    total_loss = 0
    batch_count = 0
    size = dataset.get_dataset_size()
    
    print("  开始训练循环...")
    
    # 遍历数据集
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        # 转换数据格式（与文档一致）
        data_tensor = Tensor(data[:, np.newaxis].T, ms.float32)
        label_tensor = Tensor(label[:, np.newaxis], ms.float32)
        
        # 执行训练步骤
        loss = train_step(data_tensor, label_tensor)
        
        # 累加损失
        total_loss += loss.asnumpy()
        batch_count += 1
        
        # 每100个批次打印一次损失（与文档一致）
        if batch % 100 == 0:
            loss_value = loss.asnumpy()
            print(f"loss: {loss_value:>7f}  [{batch:>3d}/{size:>3d}]")
    
    # 计算平均损失
    avg_loss = total_loss / batch_count
    return avg_loss

print("✅ 训练函数定义完成！")

🏋️ 定义模型训练函数
✅ 训练函数定义完成！


In [39]:
# ==================== 第十二部分：定义测试函数 ====================
print("🧪 定义模型测试函数")
print("="*50)

def test_loop(model, dataset, loss_fn):
    """
    测试循环函数 - 修正回归任务的评估
    参数：
        model: 神经网络模型
        dataset: 测试数据集
        loss_fn: 损失函数
    返回：
        平均损失
    """
    # 设置为评估模式
    model.set_train(False)
    
    total_loss = 0
    count = 0
    
    # 遍历测试数据集
    for data, label in dataset.create_tuple_iterator():
        # 转换数据格式（与训练一致）
        data_tensor = Tensor(data[:, np.newaxis].T, ms.float32)
        label_tensor = Tensor(label[:, np.newaxis], ms.float32)
        
        # 前向传播（预测）
        pred = model(data_tensor)
        
        # 计算损失
        loss = loss_fn(pred, label_tensor)
        total_loss += loss.asnumpy()
        count += 1
    
    # 计算平均损失
    avg_loss = total_loss / count
    return avg_loss

print("✅ 测试函数定义完成！")

🧪 定义模型测试函数
✅ 测试函数定义完成！


In [43]:
# ==================== 第十三部:训练模型 ====================
print("🚀 开始训练模型")
print("="*50)

# 设置训练轮数
epochs = 10

print(f"训练配置：")
print(f"  训练轮数：{epochs}")
print(f"  训练样本数：{dataset_train.get_dataset_size()}")
print(f"  测试样本数：{dataset_test.get_dataset_size()}")
print(f"  开始时间：{time.strftime('%Y-%m-%d %H:%M:%S')}")
print("-" * 50)

# 训练历史记录
train_loss_history = []  # 记录每个epoch的训练损失
test_loss_history = []   # 记录每个epoch的测试损失

# 开始训练循环
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}")
    print("-" * 30)
    
    # 训练阶段
    train_loss = train_loop(network, dataset_train, net_loss, net_opt)
    train_loss_history.append(train_loss)
    
    # 测试阶段
    test_loss = test_loop(network, dataset_test, net_loss)
    test_loss_history.append(test_loss)
    
    # 打印当前epoch结果
    print(f" 训练损失: {train_loss:>8f}")
    print(f" 测试损失: {test_loss:>8f}")

print("\n" + "=" * 50)
print("🎉 模型训练完成！")
print(f"结束时间：{time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"最终训练损失：{train_loss_history[-1]:.4f}")
print(f"最终测试损失：{test_loss_history[-1]:.4f}")

🚀 开始训练模型
训练配置：
  训练轮数：10
  训练样本数：314
  测试样本数：78
  开始时间：2025-12-10 15:11:03
--------------------------------------------------

Epoch 1
------------------------------
  开始训练循环...
loss: 3.988396  [  0/314]
loss: 4.194922  [100/314]
loss: 10.996804  [200/314]
loss: 92.740524  [300/314]
 训练损失: 8.194653
 测试损失: 7.126061

Epoch 2
------------------------------
  开始训练循环...
loss: 1.291683  [  0/314]
loss: 0.906311  [100/314]
loss: 13.269925  [200/314]
loss: 9.124539  [300/314]
 训练损失: 7.985721
 测试损失: 9.685973

Epoch 3
------------------------------
  开始训练循环...
loss: 2.334242  [  0/314]
loss: 1.757432  [100/314]
loss: 13.804032  [200/314]
loss: 1.178879  [300/314]
 训练损失: 8.033397
 测试损失: 7.348125

Epoch 4
------------------------------
  开始训练循环...
loss: 2.825895  [  0/314]
loss: 0.201715  [100/314]
loss: 2.630815  [200/314]
loss: 3.615944  [300/314]
 训练损失: 8.107369
 测试损失: 6.838921

Epoch 5
------------------------------
  开始训练循环...
loss: 0.904804  [  0/314]
loss: 1.322302  [100/314]
loss: 0.622319

In [44]:
# ==================== 第十四部：预测与评估 ====================
print("🔍 模型预测与评估")
print("="*50)

print("1. 预测示例：")
print("-" * 30)

# 设置模型为评估模式
network.set_train(False)

# 对前5个测试样本进行预测
for i in range(5):
    sample_data = Tensor(X_test[i:i+1], ms.float32).reshape(1, 1, -1)
    prediction = network(sample_data)
    pred_value = prediction.asnumpy()[0][0]
    
    print(f"样本 {i+1}:")
    print(f"  真实MPG值: {Y_test[i]:.2f}")
    print(f"  预测MPG值: {pred_value:.2f}")
    print(f"  误差: {abs(Y_test[i] - pred_value):.2f}")
    print()

print("\n2. 计算评估指标：")
print("-" * 30)

# 对整个测试集进行预测
all_predictions = []
for i in range(len(X_test)):
    sample_data = Tensor(X_test[i:i+1], ms.float32).reshape(1, 1, -1)
    pred = network(sample_data)
    all_predictions.append(pred.asnumpy()[0][0])

all_predictions = np.array(all_predictions)

# 计算MAE和MSE（使用numpy计算）
mae = np.mean(np.abs(Y_test - all_predictions))
mse = np.mean((Y_test - all_predictions) ** 2)

print(f"平均绝对误差(MAE): {mae:.4f}")
print(f"均方误差(MSE): {mse:.4f}")
print(f"均方根误差(RMSE): {np.sqrt(mse):.4f}")

print("\n✅ 实验完成！")

🔍 模型预测与评估
1. 预测示例：
------------------------------
样本 1:
  真实MPG值: 15.00
  预测MPG值: 13.99
  误差: 1.01

样本 2:
  真实MPG值: 10.00
  预测MPG值: 13.04
  误差: 3.04

样本 3:
  真实MPG值: 9.00
  预测MPG值: 11.56
  误差: 2.56

样本 4:
  真实MPG值: 25.00
  预测MPG值: 26.13
  误差: 1.13

样本 5:
  真实MPG值: 19.00
  预测MPG值: 20.65
  误差: 1.65


2. 计算评估指标：
------------------------------
平均绝对误差(MAE): 2.4346
均方误差(MSE): 9.9281
均方根误差(RMSE): 3.1509

✅ 实验完成！


In [ ]:
# ==================== 第十五部分：可视化训练过程 ====================
print("📈 训练过程可视化")
print("="*50)

# 创建图形
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. 训练损失曲线
axes[0, 0].plot(train_loss_history, label='训练损失', color='blue', linewidth=2)
axes[0, 0].plot(test_loss_history, label='测试损失', color='red', linewidth=2)
axes[0, 0].set_xlabel('训练轮数 (Epoch)', fontsize=12)
axes[0, 0].set_ylabel('损失值 (MSE)', fontsize=12)
axes[0, 0].set_title('训练和测试损失曲线', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()
axes[0, 0].set_xticks(range(epochs))
axes[0, 0].set_xticklabels([f'E{i+1}' for i in range(epochs)])

# 2. 损失下降速度（对数坐标）
axes[0, 1].semilogy(train_loss_history, label='训练损失', color='blue', linewidth=2, marker='o')
axes[0, 1].semilogy(test_loss_history, label='测试损失', color='red', linewidth=2, marker='s')
axes[0, 1].set_xlabel('训练轮数 (Epoch)', fontsize=12)
axes[0, 1].set_ylabel('损失值 (MSE, 对数坐标)', fontsize=12)
axes[0, 1].set_title('损失下降趋势（对数坐标）', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# 计算所有测试样本的预测值
print("正在计算预测值用于可视化...")
all_predictions = []
network.set_train(False)

for i in range(len(X_test)):
    sample_data = Tensor(X_test[i:i+1], ms.float32).reshape(1, 1, -1)
    pred = network(sample_data)
    all_predictions.append(pred.asnumpy()[0][0])

all_predictions = np.array(all_predictions)

# 3. 预测值 vs 真实值散点图
axes[1, 0].scatter(Y_test, all_predictions, alpha=0.6, color='green', s=50)
# 绘制理想预测线（y=x）
min_val = min(Y_test.min(), all_predictions.min())
max_val = max(Y_test.max(), all_predictions.max())
axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='理想预测')
axes[1, 0].set_xlabel('真实MPG值', fontsize=12)
axes[1, 0].set_ylabel('预测MPG值', fontsize=12)
axes[1, 0].set_title('预测值与真实值对比', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# 4. 误差分布直方图
errors = Y_test - all_predictions
axes[1, 1].hist(errors, bins=20, edgecolor='black', alpha=0.7, color='purple')
axes[1, 1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='零误差线')
axes[1, 1].set_xlabel('预测误差 (真实值 - 预测值)', fontsize=12)
axes[1, 1].set_ylabel('频数', fontsize=12)
axes[1, 1].set_title('预测误差分布', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

# 调整布局并显示
plt.tight_layout()
plt.show()

print("✅ 可视化完成！")